In [1]:
import pandas as pd
from os.path import join as opj
import os 
from nipype.interfaces import fsl

from glob import glob
from nilearn.input_data import NiftiMasker
from nilearn.masking import intersect_masks

from nipype.interfaces.ants import ApplyTransforms
from nipype.interfaces.afni import TStat
import subprocess

231018-14:56:21,736 nipype.utils WARNING:
	 A newer version (1.8.4) of nipy/nipype is available. You are using 1.7.0


/home/wanjiag/.local/lib/python3.6/site-packages/nilearn/datasets/__init__.py:96: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [2]:
######## Functions ########

def sh(script):
    os.system("bash -c '%s'" % script)
    
    
def mask_rois(in_file, brain_mask, file_name):
    
    mask_out_file = opj(output_dir, '{}_thre_{}_masked.nii.gz'.format(file_name,epi_mask_threshold))
    
    masked = intersect_masks([in_file, brain_mask], threshold=1, connected=False)
    masked.to_filename(mask_out_file)
    
    return mask_out_file
    
def coreg_2_epi(in_file, file_name):
    
    # Coregister any map into the epi space
    flt_out_file = opj(output_dir, '{}.nii.gz'.format(file_name))

    flt = fsl.FLIRT(in_file = in_file,
                    reference = avg_run_file,
                    apply_xfm = True,
                    uses_qform = True,
                    out_matrix_file = opj(output_dir, '{}.mat'.format(file_name)),
                    out_file = flt_out_file
                   )
    print(flt.cmdline)
    flt.run()

    # Threshold the epi space mask so it fits the original mask better and binarizes the mask
    trh_out_file = opj(output_dir, '{}_thre_{}.nii.gz'.format(file_name,epi_mask_threshold))
    trh = fsl.Threshold(thresh = epi_mask_threshold,
                        in_file = flt_out_file,
                        args = '-bin',
                        out_file = trh_out_file
                   )
    print(trh.cmdline)
    trh.run()
    
    masked_out_file = mask_rois(trh_out_file, brain_mask, file_name)
    
    return masked_out_file

def calculate_mean(in_file, file_name):
    ## Calculate mean images
    print('calculating mean......')
    #mean_img = mean_img(trim_output, target_affine=None, target_shape=None, verbose=0, n_jobs=1)
    mean_output = file_name
    #mean_img.to_filename(mean_output)
    tstat = TStat()
    tstat.inputs.in_file = in_file
    tstat.inputs.args = '-mean'
    tstat.inputs.out_file = mean_output
    print(tstat.cmdline)
    res = tstat.run() 

    print('calculating mean...output to %s'%(mean_output))



In [7]:
derivative_dir = '/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/'
preprocess_base_dir = opj(derivative_dir, 'preprocess/')
fmriprep_base_dir = opj(derivative_dir, 'fmriprep/')
automatic_detecting_subjects = True

if automatic_detecting_subjects:
    f_list = [x for x in glob(os.path.join(fmriprep_base_dir, '*sub-MONSTERA*/'))]
    subs = list(map(lambda f: f[len(os.path.commonpath(f_list))+1:-1], f_list))
    subs.sort()
    print(subs)
else:
    print(subs)

['sub-MONSTERA01', 'sub-MONSTERA02', 'sub-MONSTERA03', 'sub-MONSTERA04', 'sub-MONSTERA05', 'sub-MONSTERA06', 'sub-MONSTERA07', 'sub-MONSTERA08', 'sub-MONSTERA09', 'sub-MONSTERA10', 'sub-MONSTERA11', 'sub-MONSTERA12', 'sub-MONSTERA13', 'sub-MONSTERA14', 'sub-MONSTERA15', 'sub-MONSTERA16', 'sub-MONSTERA17', 'sub-MONSTERA18', 'sub-MONSTERA19', 'sub-MONSTERA20', 'sub-MONSTERA21', 'sub-MONSTERA22', 'sub-MONSTERA23', 'sub-MONSTERA24', 'sub-MONSTERA25', 'sub-MONSTERA26', 'sub-MONSTERA27', 'sub-MONSTERA28', 'sub-MONSTERA29', 'sub-MONSTERA31', 'sub-MONSTERA32', 'sub-MONSTERA33', 'sub-MONSTERA35', 'sub-MONSTERA36', 'sub-MONSTERA37', 'sub-MONSTERA38', 'sub-MONSTERA39', 'sub-MONSTERA40', 'sub-MONSTERA41', 'sub-MONSTERA42', 'sub-MONSTERA43', 'sub-MONSTERA44', 'sub-MONSTERA45', 'sub-MONSTERA46', 'sub-MONSTERA47', 'sub-MONSTERA48', 'sub-MONSTERA49', 'sub-MONSTERA50', 'sub-MONSTERA51', 'sub-MONSTERA52', 'sub-MONSTERA53']


In [8]:
bads = ['01', '02', '03', '04', '05', '13', '14', '20', '23', '24', '27', '30', '34']
for bad in bads:
    subs = [x for x in subs if bad not in x ]

In [12]:
subs = subs[-4:]

In [13]:
session_list = []
for i in range(1,11):
    session_list.append('task-{:02d}'.format(i))

epi_mask_threshold = 0.5

In [ ]:

for subnum in subs:
    
    print(subnum)

    output_dir = opj(derivative_dir, 'rois', subnum)

    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)

    fmriprep_dir = opj(fmriprep_base_dir, subnum)
    preprocess_dir = opj(preprocess_base_dir, subnum)

    print('--------------------{}-------------------'.format(subnum))
    print(output_dir)

    # Getting brain mask
    brain_mask = opj(preprocess_dir, '{}_space-T1w_desc-brain_intersect_mask.nii.gz'.format(subnum))

    # Finding and coverting aparc into nifti file.
    mgz_file = opj(fmriprep_base_dir, 'sourcedata', 'freesurfer', subnum, 'mri', 'aparc.a2009s+aseg.mgz')
    gii_file = opj(output_dir, 'aparc.a2009s+aseg.nii.gz'.format(subnum))
    print(mgz_file)
    print(gii_file)
    
    # Finding and copying aparcaseg file into ROIs folder
    aparc_file = os.path.join(fmriprep_dir,'anat','{}_run-5_desc-aparcaseg_dseg.nii.gz'.format(subnum))
    aparc_output = os.path.join(output_dir, 'run-5_desc-aparcaseg_dseg.nii.gz'.format(subnum))
    print(aparc_file)
    print(aparc_output)
    
    # Calculate All runs average as a reference image
    avg_run_file = opj(output_dir, 'avg_all_fmriprerp_func.nii.gz')
    
    #print('--------------------Middle Frontal Gyrus-------------------')
    #ag = {'ctx_lh_G_front_middle':11115,
    #      'ctx_rh_G_front_middle':12115}
    # good ROIs but got cut off in fMRI
    
    print('--------------------RSC-------------------')
    ag = {'ctx_lh_G_cingul-Post-dorsal':11109,
          'ctx_rh_G_cingul-Post-dorsal':12109}
    
    
    for i in ag:
        op_string = '-thr {} -uthr {}'.format(ag[i],ag[i])
        out_file = opj(output_dir, '{}.nii.gz'.format(i))

        maths = fsl.ImageMaths(in_file=gii_file, 
                                op_string=op_string,
                                out_file=out_file)
        print(maths.cmdline)
        maths.run() 
    
    in_file  = opj(output_dir, 'ctx_lh_G_cingul-Post-dorsal.nii.gz')
    op_string = '-add {} -bin'.format(opj(output_dir, 'ctx_rh_G_cingul-Post-dorsal.nii.gz'))

    out_file = opj(output_dir, 'rsc_cingul-Post-dorsal.nii.gz'.format(i))
    maths = fsl.ImageMaths(in_file=in_file, 
                            op_string=op_string,
                            out_file=out_file)
    print(maths.cmdline)
    maths.run()
    
    sh('rm {}/ctx_*_G_cingul-Post-dorsal.nii.gz'.format(output_dir))
    
    coreg_2_epi(out_file, 'rsc_cingul-Post-dorsal_2_epi')
    
    print('--------------------Orbital Gyrus-------------------')
    ag = {'ctx_lh_G_orbital':11124,
          'ctx_rh_G_orbital':12124}
    
    
    for i in ag:
        op_string = '-thr {} -uthr {}'.format(ag[i],ag[i])
        out_file = opj(output_dir, '{}.nii.gz'.format(i))

        maths = fsl.ImageMaths(in_file=gii_file, 
                                op_string=op_string,
                                out_file=out_file)
        print(maths.cmdline)
        maths.run() 
    
    in_file  = opj(output_dir, 'ctx_lh_G_orbital.nii.gz')
    op_string = '-add {} -bin'.format(opj(output_dir, 'ctx_rh_G_orbital.nii.gz'))

    out_file = opj(output_dir, 'ofc_orbital.nii.gz'.format(i))
    maths = fsl.ImageMaths(in_file=in_file, 
                            op_string=op_string,
                            out_file=out_file)
    print(maths.cmdline)
    maths.run()
    
    sh('rm {}/ctx_*_G_orbital.nii.gz'.format(output_dir))
    
    coreg_2_epi(out_file, 'ofc_orbital_2_epi')

In [14]:
#derivative_dir = '/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/'
#rois = ['angular_gyrus_2_epi_thre_0.5_masked', 'evc_2_epi_thre_0.5_masked', 'hippocampus_2_epi_thre_0.5_masked', 'ppa_mni_2_epi_thre_0.5_masked']
rois = ['rsc_cingul-Post-dorsal_2_epi_thre_0.5_masked', 'ofc_orbital_2_epi_thre_0.5_masked']

In [15]:
sublist = subs

In [17]:
sublist

['sub-MONSTERA50', 'sub-MONSTERA51', 'sub-MONSTERA52', 'sub-MONSTERA53']

In [16]:
len(sublist)

4

In [18]:
for subnum in sublist:
    
    output_dir = opj(derivative_dir,'csv_files', 'fMRI', subnum)
    
    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)

    for roi in rois:
        region_mask = opj(derivative_dir,'rois/{}/{}.nii.gz'.format(subnum, roi))
        print(roi, region_mask)
        masker = NiftiMasker(region_mask)

        for session in session_list:

            if (subnum == 'sub-MONSTERA29' and session == 'task-02'):
                continue 

            file_dir = opj(derivative_dir, 'preprocess', subnum, '{}_{}_desc-preproc_bold_trim6_denoise_z-scored.nii.gz'
                           .format(subnum, session))
            print(session, file_dir)
            region_data = masker.fit_transform(file_dir)
            
            output_file = opj(output_dir, '{}_{}_{}.csv'.format(roi, subnum, session))
            region_data = pd.DataFrame(region_data)
            region_data["run"] = session
            region_data["sub"] = subnum
            region_data["roi"] = roi
            region_data.to_csv(output_file, index=True)

rsc_cingul-Post-dorsal_2_epi_thre_0.5_masked /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/rois/sub-MONSTERA50/rsc_cingul-Post-dorsal_2_epi_thre_0.5_masked.nii.gz
task-01 /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA50/sub-MONSTERA50_task-01_desc-preproc_bold_trim6_denoise_z-scored.nii.gz
task-02 /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA50/sub-MONSTERA50_task-02_desc-preproc_bold_trim6_denoise_z-scored.nii.gz
task-03 /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA50/sub-MONSTERA50_task-03_desc-preproc_bold_trim6_denoise_z-scored.nii.gz
task-04 /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA50/sub-MONSTERA50_task-04_desc-preproc_bold_trim6_denoise_z-scored.nii.gz
task-05 /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA50/sub-MONSTERA50_task-05_desc-preproc_bold_trim6_denoise_z-scored.nii.gz
task-06 /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub